In [109]:
import pandas as pd
import numpy as np
import datetime
import math
import calendar

import folium

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import geojson
%matplotlib inline

In [110]:
html_path = '../docs/assets/htmls/'
image_path = '../docs/assets/imgs/'

In [111]:
def convert_timestring_to_time(date_string):
    if date_string is np.nan:
        return date_string
    idx = date_string.find('M')
    if( idx ==-1):
        return datetime.datetime.strptime(date_string, '%H:%M')
    idx -=2
    if date_string[idx] == ' ':
        return datetime.datetime.strptime(date_string, '%I:%M %p')
    return datetime.datetime.strptime(date_string, '%I:%M%p')

In [112]:
df = pd.read_csv("../../school-shootings-data.csv")
df['time'] = df['time'].apply(lambda x: convert_timestring_to_time(x))
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

In [113]:
df = df[df['long'].notna()]
df.reset_index(drop=True, inplace=True)

# Aggregate by time

In [7]:
# hour
df['hour'] = df['time'].dt.hour
shootings_per_hour = df.loc[:, ['hour']]
shootings_per_hour = shootings_per_hour.groupby(['hour']).aggregate(count=('hour', 'count')).reset_index()
fig = px.bar(shootings_per_hour, x="hour", y="count", height=400)
fig.update_traces(hovertemplate='Hour: %{x} <br> Incidents: %{y}')
fig.update_layout(legend_title="Incidents", 
                    xaxis_range=[0,23],
                    xaxis = dict(
                    tickmode = 'linear',
                    tick0 = 0,
                    dtick = 1
                ),
                    xaxis_title="Hour", 
                    yaxis_title="Count")

plotly.offline.plot(fig, filename=html_path+"shootings_hour.html")

'../docs/assets/htmls/shootings_hour.html'

In [9]:
# month
df['month'] = df['date'].dt.month
shootings_per_month = df.loc[:, ['month']]
shootings_per_month = shootings_per_month.groupby(['month']).aggregate(count=('month', 'count')).reset_index()
fig = px.bar(shootings_per_month, x="month", y="count", height=400)
fig.update_traces(hovertemplate='Month: %{x} <br> Incidents: %{y}')
fig.update_layout(legend_title="Incidents", 
                    xaxis = dict(
                    tickmode = 'linear',
                    tick0 = 1,
                    dtick = 1
                ),
                    xaxis_title="Month", 
                    yaxis_title="Count")

plotly.offline.plot(fig, filename=html_path+"shootings_month.html")

'../docs/assets/htmls/shootings_month.html'

In [11]:
# weekday
cat = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
shootings_per_weekday = df.loc[:, ['day_of_week']]
shootings_per_weekday = shootings_per_weekday.groupby(['day_of_week']).aggregate(count=('day_of_week', 'count')).reindex(cat)
shootings_per_weekday['weekday'] = shootings_per_weekday.index

fig = px.bar(shootings_per_weekday, x="weekday", y="count", height=400)
fig.update_traces(hovertemplate='Weekday: %{x} <br> Incidents: %{y}')
fig.update_layout(  xaxis_title="Weekday", 
                    yaxis_title="Count")
plotly.offline.plot(fig, filename=html_path+"shootings_weekday.html")

'../docs/assets/htmls/shootings_weekday.html'

# Shooter characteristic

In [5]:
races= {'a': 'Asian', 'ai':'American Indian','b':'Black','h':'Hispanic','w':'White'}
def assign_race(race):
    if race is np.nan:
        return "Not known"
    if race in races:
        return races[race]
    return "Not known"

In [6]:
df['gender_shooter1'].isna().sum()
# assign race from dictionary
df['race_ethnicity_shooter'] = df['race_ethnicity_shooter1'].apply(lambda x: assign_race(x))
# reove cells where age of the shooter was not known
df = df[df['age_shooter1'].notna()]
# convert age to int
df['age_shooter1'] = df['age_shooter1'].astype(int)
# separate data for men and women
women = df[df['gender_shooter1'] == 'f']
men = df[df['gender_shooter1'] == 'm']
# aggregate women
women_ethnicity = women.loc[:,['age_shooter1','race_ethnicity_shooter']].value_counts(dropna=False).unstack() 
women_ethnicity = women_ethnicity.fillna(0)
women_ethnicity = women_ethnicity.reset_index()
#aggregate men
men_ethnicity = men.loc[:,['age_shooter1','race_ethnicity_shooter']].value_counts(dropna=False).unstack() 
men_ethnicity = men_ethnicity.fillna(0)
men_ethnicity = men_ethnicity.reset_index()

race_ethnicity_shooter,age_shooter1,Black,Not known,White
0,12,0.0,1.0,0.0
1,14,0.0,1.0,1.0
2,15,1.0,1.0,0.0
3,22,1.0,0.0,0.0
4,28,0.0,2.0,0.0
5,30,0.0,0.0,1.0
6,33,0.0,1.0,0.0


In [46]:
marker_color = {
    "Black":"#fd7f6f",
    "Not known":"#7eb0d5",
    "White": "#b2e061",
    "Hispanic":"#bd7ebe",
    "American Indian":"#ffb55a",
    "Asian": "#ffee65",
}

In [108]:
fig = make_subplots(rows=1, cols=2, column_widths=[0.3,0.7], specs=[[{"type":"bar"}, {"type": "bar"}]], shared_xaxes=False,
                    shared_yaxes=True, horizontal_spacing=0)

base = np.zeros(shape=women_ethnicity.shape[0])  
base = pd.Series(base)
for col in women_ethnicity.columns[1:]:
    fig.append_trace(go.Bar(name="Ethincity "+col,
                        x=women_ethnicity[col],
                        y=women_ethnicity['age_shooter1'], 
                        hovertemplate='Ethinicity: ' + col + '<br>Count: ' +women_ethnicity[col].map('{:,.0f}'.format) + '<br>Age: ' + women_ethnicity['age_shooter1'].map('{:,.0f}'.format),
                        offsetgroup=0,
                        orientation='h', 
                        width=0.7, 
                        marker_color=marker_color[col],
                        showlegend=False, 
                        base=base), 
                        1, 1) # 1,1 represents row 1 column 1 in the plot grid
    base = base.add(women_ethnicity[col])


base = np.zeros(shape=men_ethnicity.shape[0])  
base = pd.Series(base)
for col in men_ethnicity.columns[1:]:
    fig.append_trace(go.Bar(name="Ethincity "+col,
                        x=men_ethnicity[col],
                        y=men_ethnicity['age_shooter1'], 
                        hovertemplate='Ethinicity: ' + col + '<br>Count: ' +men_ethnicity[col].map('{:,.0f}'.format) + '<br>Age: ' + men_ethnicity['age_shooter1'].map('{:,.0f}'.format),
                        offsetgroup=1,
                        orientation='h', 
                        width=0.7, 
                        marker_color=marker_color[col],
                        showlegend=False, 
                        base=base), 
                        1, 2) # 1,1 represents row 1 column 1 in the plot grid
    base = base.add(men_ethnicity[col])

fig.update_xaxes(showticklabels=False,title_text="Female", row=1, col=1, range=[20,0], fixedrange=True)
fig.update_xaxes(showticklabels=False,title_text="Male", row=1, col=2, fixedrange=True)

fig.update_layout(title_text="Age of the shooter with ethnicity distinguished", 
                  width=800, 
                  height=700,
                  xaxis1={'side': 'top'},
                  xaxis2={'side': 'top'},
                  margin=dict(b=50, l=50, r=50, t=150),
                  barmode='relative')

plotly.offline.plot(fig, filename=html_path+"shooter_age_gender_distribution.html")
fig.show()

# Shootings per year

In [289]:

shootings_per_year = df.groupby(['year']).agg(
     sum_killed = ('killed','sum'),
     sum_injured = ('injured','sum'),
     sum=('year', 'count')
     ).reset_index()


In [292]:
fig = go.Figure(
    data = [
        go.Bar(x=shootings_per_year['year'], y=shootings_per_year['sum_injured'], offsetgroup=0, name='People injured', hovertemplate='Year: %{x} <br> Injured: %{y}', hovertextsrc=''),
        go.Bar(x=shootings_per_year['year'], y=shootings_per_year['sum_killed'], customdata=shootings_per_year['sum_killed'], offsetgroup=0, name='People killed', hovertemplate='Year: %{x} <br> Killed: %{customdata}'),
        go.Line(x=shootings_per_year['year'], y=shootings_per_year['sum'], name='All incidents', hovertemplate='Year: %{x} <br> Incidents count: %{y}'),
    ]
)
fig.update_layout( xaxis_title="Year", yaxis_title="Count", yaxis=dict(range=[0, 95]), legend_traceorder="reversed", barmode='stack')
plotly.offline.plot(fig, filename=html_path+"shootings_year.html")

'../docs/assets/htmls/shootings_year.html'

# Plot student's ethnicity distribution in schools

In [334]:
def plot_student_race_distribution(races, column_names, title, file_path):
    races = races.dropna(axis=1)
    races = races.loc[:, (races != 0).any(axis=0)]
    names =[]
    for c in races.columns:
        names.append(column_names[c])
    fig = px.pie(values=races.values[0], names=names, title=title, height=290, width=320)
    fig.update_layout(uniformtext_minsize=6, uniformtext_mode='hide', legend=dict(
        font=dict(size=10), 
        yanchor="top",
        y=-0.01,
        xanchor="left",
        x=0.01
        ),
        margin=dict(l=20, r=20, t=20, b=20),
        font_size=8, title_font_size=10)
    fig.update_traces(hovertemplate='Ethnicity: %{label} <br>  Count: %{value}', textposition='inside')
    fig.update_traces(textfont_size=8)
    plotly.offline.plot(fig, filename=file_path, config=dict(displayModeBar=False))

In [335]:
df.reset_index(drop=True, inplace=True)

In [336]:
column_names = {'white':'White', 'black':'Black', 'hispanic':'Hispanic', 'asian':'Asian',
       'american_indian_alaska_native':'American/Indian/Alaska native', 'hawaiian_native_pacific_islander':'Hawaiian native/Pacific islander'}
df['ethnicity_diagram_path'] = np.nan
for i in range(0, df.shape[0]):
    races = df.loc[[i], ['white', 'black', 'hispanic', 'asian',
       'american_indian_alaska_native', 'hawaiian_native_pacific_islander']]
    races = races._convert(numeric=True)
    schoolname = df.iloc[i]['school_name']
    filepath = './ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    filepathtosave= html_path + 'ethinicity_diagrams/' + schoolname.replace('/', '_') +'_ethinicity.html'
    diagram_title = f"Population of {schoolname} <br>({df.iloc[i]['school_type']} school)"
    plot_student_race_distribution(races, column_names, diagram_title, filepathtosave)
    df.at[i,'ethnicity_diagram_path']= filepath

c:\Users\Marta\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1074: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

c:\Users\Marta\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1074: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

c:\Users\Marta\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1074: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

c:\Users\Marta\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1074: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a futur

# Map with student's ethnicity in school

In [188]:
def generate_color(killed, injured):
    if killed ==0 and injured==0:
        return 'blue'
    if killed ==0 :
        return 'gray'
    return 'crimson'

def calculate_circle_radius(max_affected, current_affected):
    return math.log(current_affected/ max_affected * 100 + 1.) * 2 + 4

In [357]:
colors = {'blue': 'Incident without causalities', 'gray': 'Incident with injured', 'crimson': 'Incident with killed'}
max_killed_and_injured = max(df['killed']) + max(df['injured'])
# Make an empty map
m = folium.Map(location=[40,-100], tiles="OpenStreetMap", zoom_start=2.5, height=516, width=640)
feature_groups = {}
lgd_txt = '<span style="color: {col};">{txt}</span>'
for c, t in colors.items():
   fg = folium.FeatureGroup(name=lgd_txt.format(col = c, txt=t))
   feature_groups[c] = fg

for i in range(0,df.shape[0]):
   killed_and_injured = df.iloc[i]['killed'] + df.iloc[i]['injured']
   # calculate radius
   radius = calculate_circle_radius(max_killed_and_injured, killed_and_injured)
   # prepare html text to embeed in map
   html=f"<div style='width:500px; height:300px; display: flex' > \
       <div style='flex:30%; text-align: left; margin: auto;'> \
         <span >\
            Date and time of incident: {df.iloc[i]['date'].strftime('%Y-%m-%d')}  {'' if pd.isnull(df.iloc[i]['time']) else df.iloc[i]['time'].strftime('%H:%M')}<br \> \
            Enrollment: {df.iloc[i]['enrollment']} <br \> \
            Killed/casualties: {df.iloc[i]['killed']}/{df.iloc[i]['casualties']} <br \> \
            Injured/casualties: {df.iloc[i]['injured']}/{df.iloc[i]['casualties']} <br \> <br \>\
            <\span> \
            <span> \
              Shooting details <br\> \
               Type of shooting: {df.iloc[i]['shooting_type']} <br \> \
                  Age of shooter: {'Not known' if pd.isnull(df.iloc[i]['age_shooter1']) else df.iloc[i]['age_shooter1']} \
               <\span> \
         </div> \
            <div style='flex: 70%'> \
    <iframe scrolling='no' src='{df['ethnicity_diagram_path'][i]}' width='100%' height='100%'  frameborder='0'>    \
      </div> \
    </div> \
    "
   diagram = folium.Popup(folium.Html(html, script=True), min_width=500, min_height=300, max_width=500, max_height=300)
   color = generate_color(df.iloc[i]['killed'], df.iloc[i]['injured'])
   marker = folium.CircleMarker(
      location=[df.iloc[i]['lat'], df.iloc[i]['long']],
      radius=radius,
      popup=diagram,
      color=color,
      fill=True,
      fill_color=color
   )
   feature_groups[color].add_child(marker)
for _,fg in feature_groups.items():
   m.add_child(fg)
# Save the map
folium.map.LayerControl('topleft', collapsed= True).add_to(m)
# m
m.save(html_path+'shootings_details.html')


# Gun ownership

In [358]:
file_name = '../../TL-354-State-Level Estimates of Household Firearm Ownership.xlsx'
sheet_to_df_map = pd.read_excel(file_name, sheet_name="State-Level Data & Factor Score")
gun_ownership = sheet_to_df_map.groupby(['STATE']).aggregate(
    average_ownership = ('HFR', 'mean')
).reset_index()
# convert rate to %
gun_ownership['average_ownership'] = gun_ownership['average_ownership'] * 100
# get shootings by state
school_shootings_by_state = df.groupby('state').aggregate( killed = ('killed', 'sum'), injured= ('injured', 'sum'), causalities = ('casualties', 'sum')).reset_index()

In [359]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# generate abbreviations for states
school_shootings_by_state['state_abb'] = school_shootings_by_state['state'].apply(lambda x: us_state_to_abbrev[x])
gun_ownership['state_abb'] = gun_ownership['STATE'].apply(lambda x: us_state_to_abbrev[x])
result = pd.merge(school_shootings_by_state, gun_ownership, on='state_abb', how='outer')
result['causalities']  = result['causalities'].fillna(0)
result['killed']  = result['killed'].fillna(0)
result['injured']  = result['injured'].fillna(0)
result['average_ownership']  = result['average_ownership'].fillna(0)
result['text'] = result.apply(lambda row: f"{row['state']} <br>Average % of households that own a gun: {round(row['average_ownership'],2)} % <br>Killed: {row['killed']}<br>Injured: {row['injured']}", axis=1)

In [360]:
layout = dict(font=dict(size=12), geo=dict(scope="usa", projection=dict(type="albers usa")), legend=dict(font=dict(color='black')))
data = []

data.append(go.Choropleth(locations=result['state_abb'], z=result['average_ownership'], locationmode="USA-states", colorscale="bluered", text=result["text"], hoverinfo='text', showscale=True, showlegend=False, name='Gun ownership'))
data.append(go.Scattergeo(locations=result['state_abb'], text=result['causalities'], locationmode="USA-states", mode="text", hoverinfo='skip', name='Amount of casualties', showlegend=False,textfont=dict(color='white')))

fig = go.Figure(data=data, layout=layout)
fig.update_layout(showlegend=True)

plotly.offline.plot(fig, filename=html_path+'us_gun_ownership.html')

'../docs/assets/htmls/us_gun_ownership.html'